# Práctica Métodos de Simulación - Parte 1

## Elena Rivas, Teresa Grau, Ignacio Casso

**Enunciado. Simulación de sucesos discretos.**

Un taller de fabricación se dedica a procesar tres tipos de piezas, para ello el
taller consta de cuatro células de procesamiento.

En el interior de cada célula se dispone de una máquina de procesado, excepto
en la célula 3 formada por dos máquinas con las mismas características, y de un
almacén (de capacidad ilimitada).

La secuencia de fabricación de cada una de las piezas así como los tiempos de
procesado (expresados en minutos y distribuidos según una triangular) en cada
célula se muestran en la siguiente tabla:


**TODO:** Escribir tabla usando sintáxis de hml:

| A | B | C |
| --- | --- | --- |
| a | b | c |

Los tiempos entre llegadas de las piezas al taller tiene carácter aleatorio. En el
fichero llegadas.txt se proporciona una muestra de tiempos entre llegadas de
piezas. Contrástese si la distribución de dichos tiempos es normal (truncada),
weibull o exponencial y estímense los parámetros de la distribución
correspondiente.

El fichero piezas.text incluye un histórico de piezas que han llegado al taller, que
nos permitirá identificar la proporción de piezas que hay de cada uno de los tres
tipos.Los tiempos de transporte de cada pieza entre las diferentes células es de 2
minutos.

A. Suponiendo que el taller trabaja de forma ininterrumpida (hay tres turnos de
trabajadores), simular el comportamiento del sistema durante 2 meses para
estimar el tiempo mínimo, medio y máximo que tardan en fabricarse los tres
tipos de piezas, el número medio de piezas esperando en cada una de las 4
células y la proporción de tiempo que están ociosas las máquinas de
procesado de las células.

B. Calcular las medidas anteriores si el tiempo de transporte de las piezas entre
las distintas células se reduce la mitad.

En caso de que fuese posible disponer de una máquina de procesado adicional,
¿en qué célula sería más beneficioso ponerla?

## Distribución de los tiempos de llegada

Tenemos que comprobar si los tiempos de llegada dados en el fichero llegadas.txt siguen una distribución normal truncada, weibull o exponencial, y estimar los parámetros correspondientes. Para ello podemos usar los contrastes de hipótesis vistos en ...

## Simulación del Sistema

A continuación simulamos el sistema...

Asumimos FIFO para cada celula de procesamiento...

Usamos los tiempos de llegada y las piezas de los ficheros (aunque tambien podriamos generarlos aleatoriamente, conociendo su distribucion)...
**TODO:** Usar un generador aleatorio? Implementarlo nosotros?

Usamos numpy.random.triangular para generar tiempos de procesado aleatorio...
**TODO:** Implementarlo nosotros?

En primer lugar escribimos los generadores de tiempos y piezas...

In [4]:
import numpy as np
import random
import queue
import math

In [5]:
class Piece:
    
    def __init__(self,pieceType, arrivalTime):
        
        self.arrivalTime = arrivalTime
        self.pieceType = pieceType
        self.step = 0

# TODO: are there structs in python?
# For our purposes, tuples would be enough, even if they are not mutable

In [6]:
class System():
    
    def __init__(self, transportTime, num_machines):
        self.eventsQueue = queue.PriorityQueue() # TODO: if two events occur at the exact same time,
                                                              # the priority queue will crash
        self.arrivals = open("E1.llegadas.txt","r")
        self.pieces = open("E1.piezas.txt","r")
        # open files
        
        # fabrication sequence for each type of piece
        self.steps = [[0,1,2,3],
                      [0,1,3,1,2],
                      [1,0,2]]
        
        # processing times for each type of piece at each step of its phabrication sequence
        self.processingTimes =[[(6,9,10), (5,8,10), (15,20,25), (8,12,16)],
                           [(1,13,15), (4,6,8), (15,18,21), (6,9,12), (27,30,39)], # El 30 es un 3 en el enunciado, errata
                           [(7,9,11), (7,10,13), (18,23,28)]]
        
        
        # cell queues
        self.queue = [queue.Queue(),queue.Queue(),queue.Queue(),queue.Queue()]
        
        # number of machines in each cell
        self.num_machines = num_machines
        
        # number of idle machines in each clell
        self.idle_machines = num_machines[:] # copy
        
        # transportation time between cells
        self.transportTime = transportTime
        
        # stats counters
        self.minTime = [math.inf, math.inf, math.inf] # min processing type of each type of piece
        self.maxTime = [0,0,0] # max processing type of each type of piece
        self.sumTimes = [0,0,0] # Sum of the processing times of all pieces for each type of piece
        self.numPieces = [0,0,0] # Total number of pieces processed for each type of piece
        
        self.queueSizeOverTime = [0,0,0,0] # like the area under the integral of the size of each queue over time, but discrete
        self.lastUpdate = [0,0,0,0] # Last time the previous counter was updated
        
        self.idleTime = [0,0,0,0] # total time the machine(s) in a cell have been idle
        self.lastUpdateMachine = [0,0,0,0] # Last time the previous counter was updated
        
    # destructor
        # close files

    def simul_main(self,simulationTime):
    
        self.time = 0
        
        self.simulationTime = simulationTime
        
        if simulationTime > 64500:
            print("File is not big enough, generators needed")
            return False
    
        self.nextPiece()
    
        while not self.eventsQueue.empty():
            self.time, nextEvent = self.eventsQueue.get()
            nextEvent()
            
    def simul_main_debug(self, simulationTime):
            
        self.time = 0
    
        self.simulationTime = simulationTime
    
        self.nextPiece()
            
    def debug_step(self):
        
        if not self.eventsQueue.empty():
            self.time, nextEvent = self.eventsQueue.get()
            nextEvent()
        
    def nextPiece(self):
        if self.time < self.simulationTime: # TODO: use 86400 (2 months) when we use true distribution instead of files
            nextPieceType = self.nextPieceType()
            nextArrival = self.timeTillNextArrival()
            nextArrivalTime = self.time + nextArrival
            nextPiece = Piece(nextPieceType,nextArrivalTime)
            #eventsQueue.put(self.advancePiece(nextPiece), nextArrivalTime)
            #eventsQueue.put(self.nextPiece(), nextArrivalTime)
            # The prority queue as it is breaks if two events happen at the same time
            self.eventsQueue.put((nextArrivalTime, (lambda : self.arrivePiece(nextPiece))))
    
    def arrivePiece(self,piece):
        self.nextPiece()
        self.advancePiece(piece)
        
    def nextPieceType(self):
        piece = self.pieces.readline()
        return int(piece)-1
        # Types of pieces are shifted so that the first one is 0
        # TODO: use true distribution?
        
    def timeTillNextArrival(self):
        arrival = self.arrivals.readline()
        return float(arrival)
        # TODO: use true distribution?
    
    def advancePiece(self,piece):
        
        pieceType = piece.pieceType
        step = piece.step+1
        piece.step = step
        if self.steps[pieceType][step:] == []:
            self.finishPiece(piece)
        else:
            next_cell = self.steps[pieceType][step]
            self.enter(next_cell,piece)
    
    def finishPiece(self,piece):
        
        pieceType = piece.pieceType
        
        totalTime = self.time - piece.arrivalTime
        
        if totalTime < self.minTime[pieceType]: # use dictionaries, since index needs shifting
            self.minTime[pieceType] = totalTime
       
        if totalTime > self.maxTime[pieceType]:
            self.maxTime[pieceType] = totalTime
        
        self.sumTimes[pieceType] += totalTime
        
        self.numPieces[pieceType] += 1
        
    
    def enter(self,cell,piece):
        
        if self.idle_machines[cell] > 0:
            self.process(cell,piece)
        else:
            self.queueSizeOverTime[cell] += self.queue[cell].qsize()*(self.time-self.lastUpdate[cell])
            self.lastUpdate[cell] = self.time
            self.queue[cell].put(piece)

    def process(self,cell,piece):
        
        self.idleTime[cell] += (self.time - self.lastUpdateMachine[cell])*self.idle_machines[cell]
        self.lastUpdateMachine[cell] = self.time
        
        pieceType = piece.pieceType
        step = piece.step
        
        low, mode, upp = self.processingTimes[pieceType][step]
        
        processing_time = np.random.triangular(low,mode,upp)
        
        self.idle_machines[cell] -= 1
        
        self.eventsQueue.put((self.time+processing_time, lambda : self.done(cell,piece)))
        
    def done(self,cell,piece):
        
        if self.queue[cell].empty():
            
            self.lastUpdateMachine[cell] = self.time
            self.idle_machines[cell] += 1
        
        else:
            
            self.queueSizeOverTime[cell] += self.queue[cell].qsize()*(self.time-self.lastUpdate[cell])
            self.lastUpdate[cell] = self.time
            piece = self.queue[cell].get()
            self.process(cell,piece)
        
        self.eventsQueue.put((self.time+self.transportTime, lambda : self.advancePiece(piece)))
    
    # processing time (piece)

In [17]:
system = System(2,[1,1,2,1])
system.simul_main_debug(100)

In [18]:
for _ in range(100):
    system.debug_step()

In [19]:
system.time

108.5101975

### Apartado A: Tiempo de transporte = 2 min.

In [20]:
system = System(2,[1,1,2,1])
system.simul_main(60000)

In [ ]:
system.showStats()

### Apartado B: Tiempo de transporte = 1 min.

In [ ]:
system = System(1,[1,1,2,1])
system.simulate()
system.showStats()

## Máquina de procesado adicional

Habrá que definir beneficio. Consideramos solo el tiempo?

In [ ]:
system = System(2,[2,1,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,2,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,3,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,2,2])
system.simulate()
system.showStats()